<table align="center">
 <td align="center"><a target="_blank" href="https://colab.research.google.com/github/TFMUCM01/TFM/blob/main/Analisis_sentimiento/analisis_sentimiento_noticias_es.ipynb">
        <img src="https://colab.research.google.com/img/colab_favicon_256px.png"  
        width="50" height="50" style="padding-bottom:5px;" /><br>Run in Google Colab</a></td>
 <td align="center"><a target="_blank" href="https://github.com/TFMUCM01/TFM/blob/main/Analisis_sentimiento/analisis_sentimiento_noticias_es.ipynb">
        <img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png"  
        width="50" height="50" style="padding-bottom:5px;" /><br>View on GitHub</a></td>
</table>

In [ ]:
# Paso 1: Instalación de librerías
# Se instalarán las librerías necesarias de Hugging Face y Pandas.
!pip install transformers pandas torch

import pandas as pd
import re
import io
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import drive

# PASO 1: Montar Google Drive
drive.mount('/content/drive')

In [ ]:
# PASO 2: Definir las rutas de los archivos
# Ruta del archivo original en tu Google Drive
original_file_path = '/content/drive/MyDrive/UCM/TFM/NOTICIAS_POR_ANALIZAR.csv'

In [ ]:
# PASO 3: Leer y limpiar el contenido del archivo en memoria (sin crear un archivo limpio)
print(f"Leyendo y limpiando el contenido de '{original_file_path}' en memoria...")
cleaned_lines = []
try:
    with open(original_file_path, 'r', encoding='utf-8') as infile:
        header = next(infile)
        cleaned_lines.append(header) # Guardar el encabezado

        for line in infile:
            if line.strip().startswith('"') and ',' in line[1:]:
                # Reemplazar el formato incorrecto en memoria
                line_fixed = re.sub(r'^"(\d+),""', r'\1,"', line.strip())
                line_fixed = re.sub(r'""', r'"', line_fixed)
                cleaned_lines.append(line_fixed + '\n')
            else:
                cleaned_lines.append(line)

    # Unir las líneas limpias en una sola cadena para que Pandas pueda leerlo
    cleaned_csv_string = "".join(cleaned_lines)

    # Usar `io.StringIO` para tratar la cadena como un archivo en memoria
    cleaned_csv_file = io.StringIO(cleaned_csv_string)

    # Leer el archivo virtual en un DataFrame de Pandas
    df = pd.read_csv(cleaned_csv_file)
    print(f"DataFrame cargado y limpio en memoria con {len(df)} registros.")
    print(df.head())

except FileNotFoundError:
    print(f"Error: El archivo original '{original_file_path}' no se encontró. Verifica la ruta.")
    df = None

In [ ]:
df.info()

In [ ]:
# PASO 4: Cargar el modelo para el análisis en español
if df is not None:
    # Este es el modelo para español
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print("\nModelo y tokenizer para español cargados correctamente.")

    # PASO 5: Definir la función de predicción para el modelo de 5 estrellas
    def get_sentiment_es(text):
        if not isinstance(text, str) or pd.isna(text):
            return 'N/A', np.nan, np.nan, np.nan

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        probabilities = torch.softmax(outputs.logits, dim=1)[0].cpu().numpy()

        # El modelo de nlptown mapea a 5 estrellas.
        # labels: ['1 star', '2 stars', '3 stars', '4 stars', '5 stars']
        # Mapeamos estas 5 etiquetas a 3 categorías
        predicted_class_id = np.argmax(probabilities)
        if predicted_class_id in [0, 1]:  # 1 o 2 estrellas
            sentiment = 'Negativo'
        elif predicted_class_id == 2:    # 3 estrellas
            sentiment = 'Neutral'
        elif predicted_class_id in [3, 4]:  # 4 o 5 estrellas
            sentiment = 'Positivo'
        else:
            sentiment = 'N/A'

        # Las probabilidades corresponden a las 5 estrellas
        prob_1_star = probabilities[0]
        prob_2_stars = probabilities[1]
        prob_3_stars = probabilities[2]
        prob_4_stars = probabilities[3]
        prob_5_stars = probabilities[4]

        # Calculamos las probabilidades para 3 categorías
        prob_negativa = prob_1_star + prob_2_stars
        prob_neutral = prob_3_stars
        prob_positiva = prob_4_stars + prob_5_stars

        return sentiment, prob_positiva, prob_negativa, prob_neutral

    # PASO 6: Aplicar el análisis solo a los titulares en español
    print("\nIniciando el análisis de sentimiento para titulares en español ('es')...")

    # Obtener los índices de los registros en español
    spanish_indices = df[df['IDIOMA'] == 'es'].index

    # Iterar solo sobre los registros en español
    for index in spanish_indices:
        titular = df.loc[index, 'TITULAR']
        sentiment, prob_pos, prob_neg, prob_neut = get_sentiment_es(titular)

        # Llenar las columnas de sentimiento solo para los registros en español
        df.loc[index, 'SENTIMIENTO_RESULTADO'] = sentiment
        df.loc[index, 'PROBABILIDAD_POSITIVO'] = prob_pos
        df.loc[index, 'PROBABILIDAD_NEGATIVA'] = prob_neg
        df.loc[index, 'PROBABILIDAD_NEUTRAL'] = prob_neut

    print("Análisis de sentimiento completado.")
    print("\nDataFrame con los resultados finales (primeras filas con 'es'):")
    print(df[df['IDIOMA'] == 'es'].head())

In [ ]:
print(df[['FECHA', 'TITULAR', 'sentimiento_resultado', 'probabilidad_positivo', 'probabilidad_negativa', 'probabilidad_neutral']].head())

In [ ]:
# PASO 7: Guardar el resultado final en un nuevo archivo CSV en Drive
 output_file_path = '/content/drive/MyDrive/UCM/TFM/analisis_es.csv'
df.to_csv(output_file_path, index=False)
print(f"\nResultados guardados en '{output_file_path}'.")

In [ ]:
df.info()